In [1]:
// id : ex_4.1
sealed trait Option[+A] {
    def map[B](f: A => B) : Option[B] = {
        this match {
            case Some(a) => Some(f(a))
            case None => None
        }
    }
    def flatMap[B](f: A => Option[B]) : Option[B] = {
        this match {
            case Some(a) => f(a)
            case None => None
        }
    }
    def getOrElse[B >: A](default: => B) : B = {
        this match {
            case Some(a) => a
            case None => default
        }
    }
    def orElse[B >: A](ob: => Option[B]) : Option[B] = {
        this match {
            case Some(a) => Some(a)
            case None => ob
        }
    }
    def filter(f: A => Boolean) : Option[A] = {
        this match {
            case Some(a) => if( f(a) ) { this } else { None }
            case None => None
        }
    }
}
case class Some[+A](get: A) extends Option[A]
case object None extends Option[Nothing]

None

In [2]:
// id : try ex_4.1
println(Some(3).map(_ + 1))
val NoneInt : Option[Int] = None
println(NoneInt.map(_ + 1))
println(Some(3).flatMap((e : Int) => Some("\"" + e.toString + "\"")))
println(Some(3).flatMap((e : Int) => None))
println(NoneInt.flatMap((e : Int) => Some("\"" + e.toString + "\"")))
println(Some(3).getOrElse(6))
println(None.getOrElse(6))
println(Some(3).orElse(Some(4)))
println(None.orElse(Some(4)))
println(None.orElse(None))
println(Some(3).filter(_==2))
println(Some(3).filter(_==3))
println(None.filter(_==3))

Some(4)
None
Some("3")
None
None
3
6
Some(3)
Some(4)
None
None
Some(3)
None


null

In [3]:
// id : ex_4.2
def variance(xs: Seq[Double]) : Option[Double] = {
    if (xs.isEmpty) {
        None
    } else {
        val size = xs.size
        val mean = xs.sum / size
        val sigma = xs.map((e: Double) => (math.pow(e-mean,2))).sum / size
        Some(sigma)
    }
}

variance: (xs: Seq[Double])Option[Double]


In [4]:
// id : try ex_4.2
println(variance(Seq(600,470,170,430,300)))
// http://www.mathsisfun.com/data/standard-deviation.html :)
println(variance(Seq(1,3)))
println(variance(Seq(1)))
println(variance(Seq()))

Some(21704.0)
Some(1.0)
Some(0.0)
None


null

In [5]:
def Try[A](a : => A): Option[A] = try Some(a) catch { case e: Exception => None }

Try: [A](a: => A)Option[A]


In [6]:
// id : ex_4.3

def map2[A, B, C](a: Option[A], b: Option[B])(f: (A, B) => C) : Option[C] = {
    a match {
        case None => None
        case Some(ak) => {
            b match {
                case None => None
                case Some(bk) => Try(f(ak, bk))
            }
        }
    }
}

map2: [A, B, C](a: Option[A], b: Option[B])(f: (A, B) => C)Option[C]


In [7]:
// id : try ex_4.3

def maybeFails(x : String, y : String) : Int = (x + y).toInt
println(map2(Some("12"), Some("13"))(maybeFails))
println(map2(Some("12"), Some("ab"))(maybeFails))

Some(1213)
None


null

In [8]:
// id : ex_4.4

def sequence[A](as : List[Option[A]]) : Option[List[A]] = {
    as match {
        case Nil => Some(Nil)
        case x::xs => x match {
            case None => None
            case Some(b) => map2(x, sequence(xs))(_ :: _)
        }
    }
}

sequence: [A](as: List[Option[A]])Option[List[A]]


In [9]:
// id : try ex_4.4

println(sequence(List(Some("A"), Some("B"), Some("C"))))
println(sequence(List(Some("A"), None, Some("C"))))

Some(List(A, B, C))
None


null

In [10]:
// id : ex_4.5
def traverse[A, B](as: List[A])(f: A => Option[B]) : Option[List[B]] = {
    as match {
        case Nil => Some(Nil)
        case x::xs => f(x) match {
            case None => None
            case Some(b) => map2(Some(b), traverse(xs)(f))(_ :: _)
        }
    }
}

def seq_traverse[A](as : List[Option[A]]) : Option[List[A]] = traverse(as)(x => x)

traverse: [A, B](as: List[A])(f: A => Option[B])Option[List[B]]
seq_traverse: [A](as: List[Option[A]])Option[List[A]]


In [11]:
// id : try ex_4.5

def tryToInt(i : String) = Try(i.toInt)

println(seq_traverse(List(Some("A"), Some("B"), Some("C"))))
println(seq_traverse(List(Some("A"), None, Some("C"))))
println(traverse(List("1", "2", "3"))(tryToInt))
println(traverse(List("1", "2", ""))(tryToInt))

Some(List(A, B, C))
None
Some(List(1, 2, 3))
None


null

In [12]:
// id : ex_4.6
sealed trait Either[+E, +A] {
    // def map[B1](f: B => B1): Either[A, B1] = this match {
    def map[B](f: A => B): Either[E, B] = this match {
        case Left(_) => this.asInstanceOf[Either[E,B]]
        case Right(r) => Right(f(r))
    }
    
    def flatMap[EE >: E, B](f: A => Either[EE, B]) : Either[EE, B] = this match {
        case Left(_) => this.asInstanceOf[Either[EE, B]]
        case Right(a) => f(a)
    }
    
    def orElse[EE >: E, B](b: => Either[EE, B]) : Either[EE, B] = this match {
        case Left(_) => b
        case Right(_) => this.asInstanceOf[Either[EE,B]]
    }

    def map2[EE >: E, B, C](b: Either[EE, B])(f: (A, B) => C): Either[EE, C] = this match {
        case Left(_) => this.asInstanceOf[Either[EE, C]]
        case Right(a) => {
            b match {
                case Left(_) => b.asInstanceOf[Either[EE,C]]
                case Right(bk) => Right(f(a, bk))
            }
        }
    }
}
case class Left[+E, +A](value: E) extends Either[E, A]
case class Right[+E, +A](value: A) extends Either[E, A]

defined trait Either
defined class Left
defined class Right


In [13]:
// id : try ex_4.6
val x : Either[String, Int] = Left("bad")
val y : Either[String, Int] = Right(3)
val z : Either[String, Int] = Right(-2)
println(x.map(_ + 1))
println(y.map(_+1))
def checkGood(x : Int) : Either[String, Int] = {
    if( x >= 0 ) {
        Right(x+1)
    } else {
        Left("Super bad")
    }
}
println(x.flatMap(checkGood))
println(y.flatMap(checkGood))
println(z.flatMap(checkGood))
println(x.orElse(y))
println(z.orElse(y))
val add = ((x : Int , y : Int) => x + y - 1)
println(y.map2(z)(add)) // should be zero
println(y.map2(x)(add)) // should be "bad"
println(x.map2(y)(add)) // should be "bad"

Left(bad)
Right(4)
Left(bad)
Right(4)
Left(Super bad)
Right(3)
Right(-2)
Right(0)
Left(bad)
Left(bad)


null

In [14]:
// id : ex_4.7

def traverse[E, A, B](as: List[A])(f: A => Either[E, B]) : Either[E, List[B]] = {
    as match {
        case Nil => Right(Nil)
        case x::xs => f(x) match {
            case Left(e) => Left(e)
            case Right(b) => Right(b).map2(traverse(xs)(f))(_ :: _)
        }
    }
}

def sequence[E, A](as : List[Either[E, A]]) : Either[E, List[A]] = traverse(as)(x => x)

traverse: [E, A, B](as: List[A])(f: A => Either[E,B])Either[E,List[B]]
sequence: [E, A](as: List[Either[E,A]])Either[E,List[A]]


In [20]:
// id : try ex_4.7
def tryToInt(i : String) : Either[String, Int] = {
    Try(i.toInt) match {
        case None => Left("Not a numba!")
        case Some(x) => Right(x)
    }
}

val w : List[Either[String, Int]] = List(Right(1), Right(2), Right(3))
println(sequence(w))

val x : List[Either[String, Int]] = List(Right(1), Left("Bad"), Right(3))
println(sequence(x))

val y : List[String] = List("1", "2", "3")
println(traverse(y)(tryToInt))

val z : List[String] = List("1", "2", "")
println(traverse(z)(tryToInt))


Right(List(1, 2, 3))
Left(Bad)
Right(List(1, 2, 3))
Left(Not a numba!)


null